In [12]:
import numpy as np
import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import Dataset
import pickle
from utils.helper import read_py150k_code, read_file_to_string
import regex as re

fname_prefix = "/data/users/team2_capstone/code-style-probing/"

In [13]:
#uncommented prep
combined_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_uncommented_outlier.csv")
combined_df = combined_df[combined_df['uncommented_content'].notnull()]
#test_df = combined_df.query("content != uncommented_content")
#display(combined_df)
#display(test_df)
dataset = Dataset.from_pandas(combined_df).train_test_split(test_size=0.2)
print (1)

1


In [14]:
#class prep
#class_df= pd.read_csv(fname_prefix + "data/labeled_code/bq_data_outlier_no_class.csv")
#class_df['uncommented_content'] = combined_df['uncommented_content']
#class_df = class_df[class_df['uncommented_content'].notnull()]
#class_df = class_df[class_df['no_class_content'].notnull()]
#display (class_df)
#dataset = Dataset.from_pandas(class_df).train_test_split(test_size=0.2)
print (1)

1


In [15]:
"""combined_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_uncommented_outlier.csv")
docstring_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_no_docstring_outlier.csv")
docstring_df['uncommented_content'] = combined_df['uncommented_content']
docstring_df = docstring_df[docstring_df['uncommented_content'].notnull()]
docstring_df = docstring_df[docstring_df['no_docstring_content'].notnull()]
#display (docstring_df)
dataset = Dataset.from_pandas(docstring_df).train_test_split(test_size=0.2)"""

'combined_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_uncommented_outlier.csv")\ndocstring_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_no_docstring_outlier.csv")\ndocstring_df[\'uncommented_content\'] = combined_df[\'uncommented_content\']\ndocstring_df = docstring_df[docstring_df[\'uncommented_content\'].notnull()]\ndocstring_df = docstring_df[docstring_df[\'no_docstring_content\'].notnull()]\n#display (docstring_df)\ndataset = Dataset.from_pandas(docstring_df).train_test_split(test_size=0.2)'

In [16]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration, PLBartTokenizer

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
example =dataset['train']['uncommented_content']
#label = tokenizer(example, truncation=True)
#print (len(label['input_ids']))
def tokenization(example):
    return_dict = tokenizer(example['uncommented_content'], padding='max_length', truncation=True)
    #print ((example['content'][10]))
    labels = tokenizer(example['content'], padding='max_length', truncation=True).input_ids
    return_dict['labels'] = labels
    #print (return_dict.keys())
    
    return return_dict
train_dataset = dataset["train"].map(tokenization, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
train_dataset.format['type']

test_dataset = dataset["test"].map(tokenization, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.format['type']

loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/vocab.json from cache at /soe/ksmunson/.cache/huggingface/transformers/1e2aacf615bc83f25a9d748eccb762b335eee01a29ab7a8db9b8e86cc851d489.9a48c5abf25554713c6513ab01066e53569b9a2da0d6189715951cf7c6288805
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/merges.txt from cache at /soe/ksmunson/.cache/huggingface/transformers/7eaa9b856402f05e8fdd452951872ecd3c2692ea9abb86b7ab62b07e3bc5f7de.7179059568f1a130b0a79e4bac71f38545207cab0ec45ce82ca09afadb2649a3
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/added_tokens.json from cache at /soe/ksmunson/.cache/huggingface/transformers/a3e93db547e41cdd21f01826d07c5679e111b02d8e969c607611c30a6acbe191.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/special_tokens_map.json from cache at /soe/ksmunson/.cache/huggingface/transformers/5941df5e4315c5ab63

  0%|          | 0/698 [00:00<?, ?ba/s]

  0%|          | 0/175 [00:00<?, ?ba/s]

'torch'

In [17]:
print ((train_dataset[0]))
train_dataset.save_to_disk(fname_prefix + "datasets/codet5_train_uncommented_bq_padded.hf")
test_dataset.save_to_disk(fname_prefix + "datasets/codet5_test_uncommented_bq_padded.hf")
#print (combined_df[combined_df['content'] ==  "File Error"])
#combined_df.to_csv(fname_prefix + 'data/labeled_code/full_data.csv')

{'labels': tensor([    1,  5666,  1140,   203,  2080,  2527,  1930, 20650,   203,   203,
         2080,  4204,    18,  6283,  1930,  1729,    22,  1631,    16,  8275,
           22,  2620,    16,  1842,    67,  6283,   203,   203,   203,  1106,
         7766,  4870,    22,  2419,    30,   203,   565,   632,  3845,  2039,
          203,   565,  1652,  1729,    22,  1631,    12,   203,  3639,  9813,
           16,   203,  3639,  4977,   270, 19706,  1546,  3576, 20364,  3113,
          203,  3639,  3455,  9231,    33,  8381,    16,   203,   565,   262,
           30,   203,  3639,  3536,  2276,   358,  1765,  1729,   358,  8275,
         2887,   279,   585, 12123,   203,  3639,   810,   768,   273, 20650,
           12, 15056,    18,  3015,  1435,   309,  1549,    12, 15056,    16,
          609,    13,   469,  9813,    13,   203,  3639,  1765,   280,   273,
         1729,    22,  1631,    18,  2620,    22,  1631,  1435,   203,  3639,
          327,  1765,   280,    18,  6283,   768,    